# Chest X-Rays Pneumonia Classification

### Load Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import random, os
from keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19
from keras.optimizers import Adam
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from sklearn.metrics import confusion_matrix

In [ ]:
root_dir = 'chest_xray'

train_dir = os.path.join(root_dir, 'train')
val_dir = os.path.join(root_dir, 'val')
test_dir = os.path.join(root_dir, 'train')

### Visualization

In [ ]:
def show_images(img_dir, name): 
    plt.figure(figsize=(12,12))
    for i in range(9):
        ax = plt.subplot(3, 3, i+1)
        random_img = random.choice([
            x for x in os.listdir(img_dir)
            if os.path.isfile(os.path.join(img_dir, x))
        ])
        img_normal = load_img(os.path.join(img_dir, random_img))
        ax.imshow(img_normal)
        plt.title(name)
    plt.show()


In [ ]:
show_images(os.path.join(train_dir, 'NORMAL'), 'NORMAL')
show_images(os.path.join(train_dir, 'PNEUMONIA'), 'PNEUMONIA')

### Preprocess

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=16,
    class_mode='binary')

validation_generator = train_datagen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=16,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=16,
    class_mode='binary')

### Training

In [ ]:
vgg = VGG19(weights='imagenet', include_top=False)
x = vgg.output
x = GlobalAveragePooling2D()(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=vgg.input, outputs=predictions)
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n//train_generator.batch_size,
    validation_data=validation_generator, 
    validation_steps=validation_generator.n//validation_generator.batch_size,
    epochs=30)

### Evaluation

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
model.evaluate_generator(test_generator, steps=test_generator.n//test_generator.batch_size)

In [ ]:
y_pred = model.predict_generator(test_generator, steps=test_generator.n//test_generator.batch_size)
y_pred = y_pred > 0.5

cm = confusion_matrix(test_generator.classes, y_pred)
plt.figure(figsize=(14,10))
sns.heatmap(cm, cmap='Blues', linecolor='black', linewidth=1, annot=True, fmt='', xticklabels = ['PNEUMONIA', 'NORMAL'], yticklabels = ['PNEUMONIA', 'NORMAL'])
plt.show()

In [ ]:
'''
Inspiration
1. https://github.com/ushashwat/Chest-XRay-Pneumonia-Detection
'''